In [1]:
import numpy as np
from netCDF4 import Dataset
import time
import xarray as xr
import math

In [2]:
def plus_360(ang):
    new_ang = []
    k = 0
    new_ang.append(ang[0])
    for i in range(len(ang)-1):
        next_ang = ang[i+1]+k*360
        if (ang[i] > 135)&(ang[i+1] < -135):
            k = k + 1
            next_ang = ang[i+1]+k*360
        if (ang[i] < -135)&(ang[i+1] > 135):
            k = k - 1
            next_ang = ang[i+1]+k*360
        
        new_ang.append(next_ang)
    return new_ang
              

def w_PE(tau, d_embd,rmm_series):
    m = d_embd - 1
    N = len(rmm_series) - m*tau
    
    state1 = np.zeros((N,m+1))
    for i in range(m+1):
        state1[:,i] = rmm_series[tau*i:tau*i+N]
    order_list = []  
    w_list = []
    for i in range(N):
        temp = state1[i]
        order = np.argsort(np.argsort(temp))
        order_list.append(list(order))
        w = np.var(temp)
        w_list.append(w)
    
    unique_order_list = []
    w_count_list = []
    count_list = []
    k = -1
    for item in order_list:
        k = k+1
        if item not in unique_order_list:
            unique_order_list.append(item)
            w_count_list.append(w_list[k])
            count_list.append(1)
        else:
            for i in range(len(unique_order_list)):
                if item == unique_order_list[i]:
                    w_count_list[i] = w_count_list[i] + w_list[k]
                    count_list[i] = count_list[i] + 1
    w_p_list = []                
    for i in range(len(unique_order_list)):
        w_p_list.append(w_count_list[i]/np.sum(w_list))
        
    WPE = 0
    for i in range(len(w_p_list)):
        WPE = WPE - w_p_list[i]*np.log2(w_p_list[i])
        
    n_WPE = WPE/np.log2(math.factorial(m+1))
    possible = math.factorial(m+1)
    
    return n_WPE,len(unique_order_list),possible,unique_order_list,w_p_list,w_count_list,count_list
        

In [3]:
%%time
# load a dataset
nc = Dataset('WACCM_r1i1p1f1_2015-2024.nc')
rmm1 = nc['RMM1'][:]
rmm2 = nc['RMM2'][:]
amp = nc['RMMind'][:]
phase = nc['RMMphase'][:]
# computing the propagation time series
ang = []
for i in range(len(rmm1)):
    ang.append(np.degrees(np.arctan2(rmm2[i],rmm1[i])))
ang = plus_360(ang)
    
    
tau = 3
d_embd = 3


rmm1_PE = w_PE(tau, d_embd,rmm1)
rmm2_PE = w_PE(tau, d_embd,rmm2)
amp_PE = w_PE(tau, d_embd,amp)
ang_PE = w_PE(tau, d_embd,ang)

CPU times: user 339 ms, sys: 2.56 ms, total: 341 ms
Wall time: 357 ms


In [4]:
rmm1_PE
#WPE
# # of unique permutations existed
# # of possible permutations
# permutations
# the corresponding weighted occurrence freq
# the corresponding weighted counts
# the corresponding counts

(0.7130896213604492,
 6,
 6,
 [[2, 1, 0], [2, 0, 1], [1, 0, 2], [0, 1, 2], [0, 2, 1], [1, 2, 0]],
 [0.4126604606374006,
  0.04924013931677173,
  0.03879531295674718,
  0.4120778597726686,
  0.039171816947203,
  0.048054410369208114],
 [332.06461555239093,
  39.623151456561054,
  31.218282125499478,
  331.5958012350006,
  31.521251914879205,
  38.66900473139655],
 [1050, 387, 347, 1129, 344, 387])

In [5]:
rmm2_PE

(0.7302634572658374,
 6,
 6,
 [[2, 1, 0], [2, 0, 1], [1, 0, 2], [0, 1, 2], [0, 2, 1], [1, 2, 0]],
 [0.40236776175807953,
  0.04616691974411548,
  0.050688395439499274,
  0.40858114596561707,
  0.04187031351114822,
  0.05032546358153994],
 [251.43034553858575,
  28.848644665260707,
  31.674010672386743,
  255.3129462008238,
  26.163794405195095,
  31.447222914702166],
 [1058, 382, 371, 1083, 355, 395])

In [6]:
amp_PE

(0.8386061979553419,
 6,
 6,
 [[2, 0, 1], [1, 0, 2], [0, 1, 2], [0, 2, 1], [1, 2, 0], [2, 1, 0]],
 [0.07070956812463292,
  0.07450776584871215,
  0.3437445947510657,
  0.07332315994806281,
  0.07564007478044629,
  0.3620748365470799],
 [26.619599371626645,
  28.04948367767986,
  129.40742874156862,
  27.603522327258812,
  28.47575710752955,
  136.30810294926414],
 [436, 427, 959, 410, 453, 959])

In [7]:
ang_PE

(0.7106449654355322,
 6,
 6,
 [[2, 1, 0], [2, 0, 1], [0, 1, 2], [1, 2, 0], [0, 2, 1], [1, 0, 2]],
 [0.08792412939925008,
  0.0946068663349788,
  0.6146813112513705,
  0.09641746401184917,
  0.057171376691134555,
  0.04919885231141802],
 [426879.0387169345,
  459324.2882589375,
  2984329.433308985,
  468114.8921703608,
  277571.8393891021,
  238864.5633931484],
 [338, 269, 2207, 276, 272, 282])